Purpose: Function to accept user-defined LEHD input file from the FTP site and a local csv of ACS data (this script is built to use the same format as the 5-yr ACS csv download from NHGIS)

In [1]:
# Import necessary libraries
import pandas as pd

In [2]:
# Define function to retrieve and read the file from the site
def aggregate_LEHD(_state, _type, _segment, _JT_code, _year):
    urlbase = 'https://lehd.ces.census.gov/data/lodes/LODES7/'
    file_name = _state + '_' + _type + '_' + _segment + '_' + _JT_code + '_' + _year + '.csv.gz'
    file_path = urlbase + _state + '/' + _type + '/' 
    file = pd.read_csv(file_path + file_name)
    #Clip the wi_rac dataframe to include only the relevant columns: C000, CE01, CE02, CE03, h_geocode
    #wi_rac2 = wi_rac.iloc['h_geocode','C000','CE01','CE02','CE03'],[]
    file_clipped = file.iloc[:,[0,1,5,6,7]]
    # Add a column that will later hold the block group number from each h_code
    file_clipped.insert(1, 'BlkGrp_code', '0')
    # Set block group equal to the string h_geocode
    file_clipped.loc[:,'BlkGrp_code']= file_clipped.loc[:,'h_geocode'].astype(str)
    # Chop the string down to the countytractblockgroup numbers only, cutting off the block id (last 3 digits).
    file_clipped.loc[:,'BlkGrp_code']= file_clipped.loc[:,'BlkGrp_code'].str[:-3]
    # For each unique BlkGrp_code value, aggregate the rows into one representing the sum of each jobs column while taking
    # the first h_geocode to make sure the blk grp code is correct
    file_agg = file_clipped.groupby(file_clipped['BlkGrp_code']).agg({'h_geocode':'first', 'C000':'sum', 'CE01':'sum','CE02':'sum','CE03':'sum',})
    #Add columns containing information on percent low, me, high income jobs
    file_agg['CE01%']= file_agg['CE01']/file_agg['C000']
    file_agg['CE02%']= file_agg['CE02']/file_agg['C000']
    file_agg['CE03%']= file_agg['CE03']/file_agg['C000']
    
    return file_agg

In [23]:
# Function to read NHGIS downloaded file with the following conditions: block group geography, 5-year estimates, choosing the 
# total population and median hhinc in last 12 months fields:
def get_ACS(_ACS_file):
    # Read the csv from a local directory
    ACS = pd.read_csv(_ACS_file)
    # Reduce the dataframe to only the relevant columns including the blok grp code, and estimates and moe for pop and med hhinc
    ACS = ACS.loc[:, ['GISJOIN','AJWME001','AJZAE001','AJWMM001','AJZAM001']]
    # Remove leading 0 and extra 0 at position 3 in string
    ACS.loc[:,'GISJOIN']= ACS.loc[:,'GISJOIN'].str[1:3] + ACS.loc[:,'GISJOIN'].str[4:]
    # Remove extra 0 at position 7 in string
    ACS.loc[:,'GISJOIN']= ACS.loc[:,'GISJOIN'].str[:5] + ACS.loc[:,'GISJOIN'].str[6:]
    return ACS

In [34]:
def main(_state, _type, _segment, _JT_code, _year, _ACS_file):
    rac_file = aggregate_LEHD(_state,_type,_segment,_JT_code,_year)
    ACS_file = get_ACS(_ACS_file)
    merged = rac_file.merge(ACS_file, left_on= 'BlkGrp_code', right_on= 'GISJOIN')
    return merged

In [35]:
WI = main('wi', 'rac', 'S000', 'JT00', '2017', r'Wisconsin\NHGIS\nhgis0005_ds239_20185_2018_blck_grp.csv')


In [36]:
print(WI[:5])

      h_geocode   C000   CE01   CE02   CE03     CE01%     CE02%     CE03%  \
0  5.500195e+14  423.0  121.0  144.0  158.0  0.286052  0.340426  0.373522   
1  5.500195e+14  337.0  106.0   97.0  134.0  0.314540  0.287834  0.397626   
2  5.500195e+14  463.0  158.0  135.0  170.0  0.341253  0.291577  0.367171   
3  5.500195e+14  623.0  189.0  238.0  196.0  0.303371  0.382022  0.314607   
4  5.500195e+14  322.0   98.0  122.0  102.0  0.304348  0.378882  0.316770   

        GISJOIN  AJWME001  AJZAE001  AJWMM001  AJZAM001  
0  550019501001       875   59239.0       147    5351.0  
1  550019501002       672   51250.0       155   18399.0  
2  550019501003      1323   53684.0       182    9265.0  
3  550019502011      1244   41853.0        96    3220.0  
4  550019502021       776   40250.0       100    6648.0  


In [24]:
WI_acs = get_ACS(r'Wisconsin\NHGIS\nhgis0005_ds239_20185_2018_blck_grp.csv')
print(WI_acs[:5])

        GISJOIN  AJWME001  AJZAE001  AJWMM001  AJZAM001
0  550019501001       875   59239.0       147    5351.0
1  550019501002       672   51250.0       155   18399.0
2  550019501003      1323   53684.0       182    9265.0
3  550019502011      1244   41853.0        96    3220.0
4  550019502021       776   40250.0       100    6648.0


In [12]:
WI_LEHD = aggregate_LEHD('wi', 'rac', 'S000', 'JT00', '2017')
print(WI_LEHD[:5])

                    h_geocode  C000  CE01  CE02  CE03     CE01%     CE02%  \
BlkGrp_code                                                                 
550019501001  550019501001000   423   121   144   158  0.286052  0.340426   
550019501002  550019501002000   337   106    97   134  0.314540  0.287834   
550019501003  550019501003001   463   158   135   170  0.341253  0.291577   
550019502011  550019502011000   623   189   238   196  0.303371  0.382022   
550019502021  550019502021000   322    98   122   102  0.304348  0.378882   

                 CE03%  
BlkGrp_code             
550019501001  0.373522  
550019501002  0.397626  
550019501003  0.367171  
550019502011  0.314607  
550019502021  0.316770  
